In [12]:
import pandas as pd
import numpy as np

In [14]:
# Loading wine synonyms input data from file created with unique grape for one or several wines
syns_raw = pd.read_csv("../raw_data/wine_synonyms_csv.csv", index_col=0)

# syns_raw.head()

,NAME_ORIGINAL,NAME_SYNONYMS,NAME_DIRTY,NAME_ALL,TYPE,Blend
ID_original_Order,,,,,,
w003,Abouriou,NaN,NaN,Abouriou,Red,single_var
w004,Agiorgitiko,St. George,NaN,"Agiorgitiko, St. George",Red,single_var
w005,Aglianico,NaN,NaN,Aglianico,Red,single_var
w006,Aidani,NaN,NaN,Aidani,White,single_var
w007,Airen,NaN,NaN,Airen,White,single_var


In [16]:
# Extracting the single column from the synonyms files that has all synonyns for each row 
all_grape_names = syns_raw.NAME_ALL.str.split(', ')

# The synonyms file has multiple rows (synonyms) that hasn't been unified (synonymised) properly.
# Here a table of synonyms is created to consolidate all synonyms.
syns = {}     # for each item: key will be main synonym, values will include all synonyms including the main one (used for key)
for row in all_grape_names:
    flat_dict = [num for elem in list(syns.values()) for num in elem]
    # checking if synonyms in each row are already present is the dictionary being created
    # if not, it creates a key and values
    if any(grape in flat_dict for grape in row) == False:
        syns[row[0]] = row
    # if yes, adds the new synonyns that don't exist yet in the list
    else:
        res = next((sub for sub in syns if any(grape in syns[sub] for grape in row) == True), None)
        syns[res].extend([item for item in row if item not in syns[res]])

del flat_dict   # very long list, better to delete it to release memory
# syns

In [17]:
# Loading main wine data
data = pd.read_csv("../raw_data/wine_data_csv.csv", index_col=0)
# data.head()

In [18]:
# Using the unified synonym table, it populates the column for grape vaiety in the main data table
# with the main synonyms for each grape variety
def match_grape(grape):
    for key, value in syns.items():
        if grape in value:
            return key
data['variety_adj'] = data['variety'].apply(match_grape)

In [19]:
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,variety_adj,type,region
ID,,,,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin Oâ€™Keefe,@kerinokeefe,Nicosia 2013 VulkÃ Bianco (Etna),White Blend,Nicosia,White Blend,white,Etna
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Portuguese Red,red,Douro
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwineÂ,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Pinot Gris,white,Willamette Valley
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Riesling,white,Lake Michigan Shore
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwineÂ,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Pinot Noir,red,Willamette Valley
